In [10]:
#model accuracy very low:  Test Accuracy: 0.3369
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# Load the Reduced FER2013 dataset
train_data = pd.read_csv("DATA/Reduced_train.csv")
test_data = pd.read_csv("DATA/Reduced_test.csv")
full_data = pd.read_csv("DATA/Reduced_fer20131.csv")

# Function to preprocess pixel data
def preprocess_pixels(pixels):
    # Convert space-separated string into a numpy array
    array = np.array([int(p) for p in pixels.split()], dtype=np.uint8)
    return array.reshape(48, 48, 1)  # FER2013 images are 48x48 pixels, grayscale

# Preprocess training data
X_train = np.array([preprocess_pixels(row) for row in train_data["pixels"]])
y_train = to_categorical(train_data["emotion"].values, num_classes=7)

# Preprocess full data for validation/testing
X_full = np.array([preprocess_pixels(row) for row in full_data["pixels"]])
y_full = to_categorical(full_data["emotion"].values, num_classes=7)

# Split the full dataset into validation and test sets based on the `Usage` column
validation_mask = full_data["Usage"] == "PublicTest"
test_mask = full_data["Usage"] == "PrivateTest"

X_val, y_val = X_full[validation_mask], y_full[validation_mask]
X_test, y_test = X_full[test_mask], y_full[test_mask]

print("Training samples:", X_train.shape[0])
print("Validation samples:", X_val.shape[0])
print("Test samples:", X_test.shape[0])

Training samples: 2872
Validation samples: 385
Test samples: 377


In [11]:
# Define the CNN model
model = Sequential([
    Input(shape=(48, 48, 1)),  # Define input shape explicitly
    Conv2D(64, (3, 3), activation="relu", padding="same"),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPool2D((2, 2)),
    Dropout(0.25),

    Flatten(),
    Dense(256, activation="relu", kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(7, activation="softmax")  # 7 output classes for emotions
])

# Compile the model
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

In [12]:
# Save the best model during training
checkpoint = ModelCheckpoint("model_checkpoint.keras", save_best_only=True, monitor="val_loss", verbose=1)

# Stop training early if validation loss stops improving
early_stopping = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)

In [13]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=30,  # Reduced from 45 for time optimization
    callbacks=[checkpoint, early_stopping, reduce_lr],
    verbose=1)

Epoch 1/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - accuracy: 0.2243 - loss: 7.2843  
Epoch 1: val_loss improved from inf to 23.77011, saving model to model_checkpoint.keras
90/90 ━━━━━━━━━━━━━━━━━━━━ 26s 261ms/step - accuracy: 0.2245 - loss: 7.2510 - val_accuracy: 0.1610 - val_loss: 23.7701 - learning_rate: 0.0010
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.2616 - loss: 2.5824 
Epoch 2: val_loss improved from 23.77011 to 15.83511, saving model to model_checkpoint.keras
90/90 ━━━━━━━━━━━━━━━━━━━━ 28s 308ms/step - accuracy: 0.2614 - loss: 2.5819 - val_accuracy: 0.1714 - val_loss: 15.8351 - learning_rate: 0.0010
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.2333 - loss: 2.4353 
Epoch 3: val_loss improved from 15.83511 to 3.46714, saving model to model_checkpoint.keras
90/90 ━━━━━━━━━━━━━━━━━━━━ 25s 282ms/step - accuracy: 0.2336 - loss: 2.4351 - val_accuracy: 0.1974 - val_loss: 3.4671 - learning_rate: 0.0010
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━

In [14]:
# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.2688 - loss: 2.1154
Test Loss: 2.0460
Test Accuracy: 0.3369


In [16]:
# Save locally and download
model.save("reduced_fer2013_model.keras")